In [1]:
%load_ext autoreload

In [2]:
from linear_models import LinearOneVsAllClassifier
import numpy as np
import noise_functions_multi as noise
import time
import ray
from utils import find_noise_bounds_multi
%autoreload 2

In [3]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:50054 to respond...
Waiting for redis server at 127.0.0.1:37417 to respond...
Starting local scheduler with the following resources: {'GPU': 0, 'CPU': 4}.

View the web UI at http://localhost:8896/notebooks/ray_ui1239.ipynb?token=f9780efb8bf53b4125b1ddc8a8b0eb701406afa73ec2e858



{'local_scheduler_socket_names': ['/tmp/scheduler95728787'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store87254246', manager_name='/tmp/plasma_manager69738534', manager_port=56048)],
 'redis_address': '127.0.0.1:50054',
 'webui_url': 'http://localhost:8896/notebooks/ray_ui1239.ipynb?token=f9780efb8bf53b4125b1ddc8a8b0eb701406afa73ec2e858'}

In [4]:
linear_models = [LinearOneVsAllClassifier(10, np.load('linear_models/multi/w_{}.npy'.format(i)), np.load('linear_models/multi/b_{}.npy'.format(i)))
                 for i in range(5)]

In [5]:
X_exp = np.load('linear_experiments_data/multi/X_exp.npy')
Y_exp = np.load('linear_experiments_data/multi/Y_exp.npy')

In [ ]:
linear_models[0].predict(X_exp[:1])

In [14]:
s = time.time()
v = noise.grad_desc_convex.remote([1], linear_models[:1], X_exp[0], Y_exp[0], .2)
v = ray.get(v)
print time.time() - s 

0.468487977982


In [ ]:
print v.shape
print np.linalg.norm(v)

In [ ]:
linear_models[0].predict((X_exp[0] + v).reshape(1,-1))

In [ ]:
mnist_images

In [ ]:
find_noise_bounds_multi(linear_models[:1], X_exp[:1])

In [ ]:
np.linalg.norm(v[1])

In [ ]:
s = time.time()
results = [noise.grad_desc_nonconvex.remote([1,1,1,1,1], linear_models, X_exp[i], Y_exp[i], 1) for i in xrange(500)]
res = ray.get(results)
print time.time() - s 

In [ ]:
s = time.time()
results = [noise.grad_desc_nonconvex([1,1,1,1,1], linear_models, X_exp[i], Y_exp[i], 1) for i in xrange(500)]
print time.time() - s 

In [ ]:
[model.evaluate((X_exp[0] + v).reshape(1,-1), Y_exp[0]) for model in linear_models]